In [1]:
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'

[sudo] password for dlinano: 


In [1]:
import cv2

In [2]:
print(cv2.__version__)

4.1.1


In [4]:
# from jetcam.csi_camera import CSICamera

# camera0 = CSICamera(capture_device=0, width=224, height=224)
# camera1 = CSICamera(capture_device=1, width=224, height=224)

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

In [6]:
image=camera.read()
print(image.shape)


(224, 224, 3)


In [7]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')
cv2_image_widget=ipywidgets.Image(format='jpeg')
gray_image_widget=ipywidgets.Image(format='jpeg')
edge_image_widget=ipywidgets.Image(format='jpeg')
    
gray_image_widget.value=bgr8_to_jpeg(image)
edge_image_widget.value=bgr8_to_jpeg(image)    
image_widget.value = bgr8_to_jpeg(image)
cv2_image_widget.value=bgr8_to_jpeg(image)

all_widget = ipywidgets.VBox([
    ipywidgets.HBox([image_widget,gray_image_widget]),
     ipywidgets.HBox([cv2_image_widget,edge_image_widget])
])
display(all_widget)

In [17]:
def detect(frame):
    #加载人脸识别的XML分类器
    cascPath = "/home/dlinano/opencv/data/haarcascades/haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    #将传入的图像转换为灰度模式
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #通过gray_image_widget展示灰度图像
#     gray_image_widget.value = bgr8_to_jpeg(gray)
    #执行人脸识别，detectMultiScale函数的详细说明见教材85页“多尺度检测”
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5)
    #打印识别到的人脸数量并绘画到传入的图像中
#     print("Found {0} facse!".format(len(faces)))
    cv2.putText(frame,"Found {0} facse!".format(len(faces)),(0,14),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
    return gray

In [9]:
def edge(frame):
    edges = cv2.Canny(frame, 100, 200)
    return edges

In [10]:
camera.running = True

def update_image(change):
    image = change['new']
    edges=edge(image)
    edge_image_widget.value=bgr8_to_jpeg(edges)
    image_widget.value = bgr8_to_jpeg(image)
    gray=detect(image)
    cv2_image_widget.value=bgr8_to_jpeg(image)
    gray_image_widget.value=bgr8_to_jpeg(gray)
    
    
camera.observe(update_image, names='value')

In [ ]:
camera.unobserve(update_image, names='value')